In [18]:
import numpy 
numpy.random.seed(1337)

In [19]:
import h5py
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D, Dropout
from keras.layers import Conv2D, Embedding
from keras.optimizers import SGD, rmsprop
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras_contrib.layers.advanced_activations import PELU, SReLU

In [40]:
batch_size=128 
num_classes=37
epochs=60

In [21]:
import os
from PIL import Image
import matplotlib.image as mpimg

In [22]:
os.chdir("E:");
path="E:/Dummy/Level_1";
classes=os.listdir(path)
x=[]#Datapoints 
y=[]#labels 
for fol in classes:
    imgfiles=os.listdir(path+u'\\'+fol);
    for img in imgfiles:
        im=mpimg.imread(path+u'\\'+fol+u'\\'+img);
        x.append(im)
        y.append(fol)
x=numpy.array(x)
y=numpy.array(y)

In [23]:
print(x.shape)
print(y.shape)
x=x.reshape((-1,100,100,1))

(1080, 100, 100)
(1080,)


In [24]:
n=x.shape[0]
randomize=numpy.arange(n)
numpy.random.shuffle(randomize)
randomize
x=x[randomize]
y=y[randomize]

In [25]:
test_split=round(n*2/3)
x_train=x[:test_split]
y_train=y[:test_split]
x_test=x[test_split:]
y_test=y[test_split:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(720, 100, 100, 1)
(720,)
(360, 100, 100, 1)
(360,)


In [26]:
y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test, num_classes)
y_train[:1]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [34]:
# inputs=Input(shape=(100,100,1))
# x=Conv2D(32, (3, 3), padding='same')(inputs)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)
# x=Conv2D(7, (3, 3), padding='same')(x)
# x=SReLU()(x)
# x=MaxPooling2D(pool_size=(2,2))(x)

# x=Dropout(0.2)(x)#---------------------------------------------->0.3389(Epoch_24)
# x=Dense(num_classes)(x)
# x=SReLU()(x)
# x=Flatten()(x)
# x=Dense(num_classes)(x)

# output=Activation('softmax')(x)
# model=Model([inputs], output)

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 100, 32)      320       
_________________________________________________________________
s_re_lu_4 (SReLU)            (None, 100, 100, 32)      1280000   
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 50, 50, 7)         2023      
_________________________________________________________________
s_re_lu_5 (SReLU)            (None, 50, 50, 7)         70000     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 25, 25, 7)         0         
__________

In [36]:
opt=keras.optimizers.Adam(lr=1e-3)

In [37]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [38]:
import h5py
from keras.callbacks import ModelCheckpoint
filepath='E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-{epoch:2d}-{val_acc:.2f}.hdf5'
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')

In [41]:
hist=model.fit(x_train, y_train, 
               batch_size=batch_size,
               epochs=epochs,
               validation_data=(x_test, y_test),
               callbacks=[checkpoint]
               )

Train on 720 samples, validate on 360 samples
Epoch 1/60
720/720 [==============================] - 40s 55ms/step - loss: 3.4434 - acc: 0.1417 - val_loss: 3.2818 - val_acc: 0.1417

Epoch 00001: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement- 1-0.14.hdf5
Epoch 2/60
720/720 [==============================] - 38s 52ms/step - loss: 2.9946 - acc: 0.2806 - val_loss: 2.8090 - val_acc: 0.2167

Epoch 00002: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement- 2-0.22.hdf5
Epoch 3/60
720/720 [==============================] - 36s 50ms/step - loss: 2.3828 - acc: 0.3528 - val_loss: 2.2351 - val_acc: 0.2750

Epoch 00003: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement- 3-0.28.hdf5
Epoch 4/60
720/720 [==============================] - 35s 49ms/step - loss: 1.8520 - acc: 0.3833 - val_loss: 1.9944 - val_acc: 0.3000

Epoch 00004: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/

720/720 [==============================] - 35s 49ms/step - loss: 0.6961 - acc: 0.6694 - val_loss: 2.3360 - val_acc: 0.3139

Epoch 00034: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-34-0.31.hdf5
Epoch 35/60
720/720 [==============================] - 35s 49ms/step - loss: 0.6550 - acc: 0.6875 - val_loss: 2.3590 - val_acc: 0.3083

Epoch 00035: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-35-0.31.hdf5
Epoch 36/60
720/720 [==============================] - 35s 49ms/step - loss: 0.6743 - acc: 0.6889 - val_loss: 2.4457 - val_acc: 0.3139

Epoch 00036: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-36-0.31.hdf5
Epoch 37/60
720/720 [==============================] - 35s 49ms/step - loss: 0.6626 - acc: 0.6722 - val_loss: 2.4356 - val_acc: 0.3000

Epoch 00037: saving model to E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_24/weights-improvement-37-0.30.hdf5
Epoch 38/60
